In [1]:
from comet_ml import Experiment

In [4]:
from comet_ml import Experiment
experiment = Experiment(
  api_key = "gFC2BZfeX7KSnZwINu6BF7IcJ",
  project_name = "explore-hackathon",
  workspace="deolabeeb"
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/deolabeeb/adv-classification/fc41b32441a04c6984f29bf9ea8fda4b
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     git metadata                 : 1
COMET INFO:     git-patch (uncompressed)     : 1 (62.91 KB)
COMET INFO:     installed packages           : 1
COMET INFO:     notebook                     : 1
COMET INFO:     source_code                  : 1
COMET INFO: 
COMET WARNING: As you are running in 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk
import joblib as jb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, confusion_matrix,
                             ConfusionMatrixDisplay, roc_curve, f1_score, roc_auc_score,
                             classification_report)
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [6]:
df_train = pd.read_csv("data/cleaned_data.csv")

In [7]:
df_train.head()

,lang_id,text
0,xho,umgaqosiseko wenza amalungiselelo kumaziko axh...
1,xho,dha iya kuba nobulumko bokubeka umsebenzi naph...
2,eng,province kwazulunatal department transport inv...
3,nso,netefatša gore file dilo moka tše dumelelanego...
4,ven,khomishini ndinganyiso mbeu ewa maana nga mula...


In [8]:
X = df_train['text']
y = df_train['lang_id']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Convert text to TF-IDF features
    ('svm', SVC())  # Support Vector Classifier
])

In [12]:
param_grid = {
    'tfidf__max_features': [1000, 5000, 10000],  # Maximum number of features in TF-IDF
    'svm__C': [0.1, 1, 10],  # Regularization parameter
    'svm__kernel': ['linear', 'rbf']  # Kernel type
}

In [13]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('svm', SVC())]),
             param_grid={'svm__C': [0.1, 1, 10],
                         'svm__kernel': ['linear', 'rbf'],
                         'tfidf__max_features': [1000, 5000, 10000]})

In [14]:
accuracy = grid_search.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9931818181818182


In [15]:
best_params = grid_search.best_params_
print("Best parameters:", best_params)

Best parameters: {'svm__C': 10, 'svm__kernel': 'rbf', 'tfidf__max_features': 10000}


In [16]:
# Set the best parameters
best_params = {'svm__C': 10, 'svm__kernel': 'rbf', 'tfidf__max_features': 10000}

# Create a pipeline with the best parameters
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=best_params['tfidf__max_features'])),
    ('svm', SVC(C=best_params['svm__C'], kernel=best_params['svm__kernel']))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred_test = pipeline.predict(X_test)

### Evaluate the test data

In [17]:
df_test = pd.read_csv("data/cleaned_test_data.csv")

In [18]:
df_test.head()

,index,text
0,1,mmasepala maemo kgethegileng letlelela kgato
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,tshivhumbeo tshi fana ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini...
4,5,winste buitelandse valuta


In [19]:
# Convert test data into numerical representation
X_test_final = vectorizer.transform(df_test['text'])

NameError: name 'vectorizer' is not defined

In [ ]:
df_sample = pd.read_csv("data/sample_submission.csv")
df_sample.head()

In [ ]:
df_test['index'] = df_test.index + 1  # Add 1 to the index to match the desired format
submission_df = df_test[['index']].copy()  # Select the 'index' column
submission_df['lang_id'] = y_pred_test  # Add 'lang_id' column with predicted values

# Rename the columns as per the format
submission_df.columns = ['index', 'lang_id']

# Save the submission DataFrame to a CSV file
submission_df.to_csv('data/submission_df.csv', index=False)

In [ ]:
submission_df

In [ ]:
# Save the trained model
joblib.dump(model, 'data/trained_SVC_model.pkl')

COMET ERROR: Heartbeat processing error
